# Avaliação de Medidas (In)Coerência do conjunto de dados CSTNews usando BERT Pré-treinado

Este notebook, realiza testes de medidas de (in)coerência entre pares de documentos do conjunto de dados CSTNews 5.0 modificado por Marcio Dias analisar a coerência de sumários multidocumentos utilizando BERT Pré-treinado. 

O conjunto de dados foi construído a partir de 251 documentos oriundos do córpus CSTNews 5.0. O CSTNews é formado por textos jornalisticos de domínos váriados produzidos por (Aleixo e Pardo, 2008). Somente postagens que possuem de 4 sentenças foram selecionadas. Pois é necessário no mínimo 4 sentenças para gerar 20 permutações. 
Dois documentos(C3_Extrato_6 e C33_Extrato_6) foram removidos do conjunto de dados apesar de estarem na faixa de 4 a 20 sentenças, pois ao a realizar a formatação para a entrada no BERT eles geram mais de 512 tokens.    

Para cada documento as sentenças foram permutadas em até 20 vezes para um criar um conjunto de sentenças formadas por estas permutações.

Ao final a base de dados é formada por pares de documento, sendo um documento original(DO) e uma de suas 20 permutações(permDoc). O conjunto de dados total é formado por 4.980 pares de documentos. 

A medida de (in)coerência **Cm(D)** de um documento(D) é define a coerência e incoerência como a média de alguma função **m** de distância ou similaridade, respectivamente, entre pares de sentenças (**Si e Sj**). A função **Scos** utiliza a similaridade do cosseno e as funções **Seuc** e **Sman** utilizam distância Euclidiana e Manhattan entre os embeddings de sentenças adjacentes (**Si**) e (**Sj**) no documento.

As seguintes funções de similaridade e distância foram calculadas entre os embeddings das sentenças **Si** e **Sj**:
- **Scos(Si,Sj)** - Similaridade do cosseno entre a média dos embeddings Si e Sj.
- **Seuc(Si,Sj)** - Distância Euclidiana(L2) - usando a média dos embeddings Si e Sj das camadas especificadas
- **Sman(Si,Sj)** - Distância de Manhattan(L1) - usando a média dos embeddings Si e Sj das camadas especificadas

Características dos testes:
 - A avaliação das medidas de (in)coerência são realizadas utilizando a implementação BERT da biblioteca Huggingface e os MCL BERT **Pré-treinado** no formato cased:
  - BERTimbau de tamanho Large 
  - BERTimbau de tamanho Base 
  - BERT Multilingue
- Não é realizado ajuste fino do modelo.
- Todo o documento é submetido ao BERT e os embeddings da concatenação das 4 últimas camadas é recuperado.

Utilizando o MCL BERT multilingual os documentos C33_Extrato_6.txt, C3_Extrato_6.txt e C27_Extrato_6.txt possuem mais de 512 tokens.

Utilizando o MCL BERTimbau base e large os documentos C33_Extrato_6.txt e C3_Extrato_6.txt por possuir mais de 512 tokens.

----------------------------
**CSTNews 5.0:** https://sites.icmc.usp.br/taspardo/sucinto/files/CSTNews%205.0.zip

**Trabalho Marcio Dias:** https://sites.icmc.usp.br/taspardo/Summary%20coherence%20models.zip

**Link biblioteca Transformers:**
https://github.com/huggingface/transformers

**Artigo original BERT:**
https://arxiv.org/pdf/1506.06724.pdf

# 1 Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## 1.1 Tempo inicial de processamento

In [ ]:
import time
import datetime

#marca o tempo de início do processamento.
inicioProcessamento = time.time()

## 1.2 Tratamento de logs

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 1.3  Identificando o ambiente Colab

In [ ]:
# Se estiver executando no Google Colaboratory.
import sys

# Retorna true ou false se estiver no Google Colaboratory.
IN_COLAB = 'google.colab' in sys.modules

## 1.4 Biblioteca de limpeza de tela

In [ ]:
from IPython.display import clear_output

## 1.5 Conecta ao Google Drive

Usa a pasta mapeada para recuperar o conjunto de dados do OnlineEduc e salvar os resultados do cálculo de medidas.

In [ ]:
 # Import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount('/content/drive')     

Mounted at /content/drive


## 1.6 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 139 kB 33.6 MB/s 
     |████████████████████████████████| 97 kB 5.7 MB/s 
     |████████████████████████████████| 180 kB 47.0 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=fbb8c67c9b63a5b40f87bdcefe8ff88084c7bee720e5ac8439bc799ac9f0653e
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=df74efda1d85ac644cf91232373dc2a083cebfd2a93cb563b439b5935906a27c
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


## 1.7 Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [ ]:
!pip install -U transformers==4.5.1

     |████████████████████████████████| 2.1 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 37.6 MB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 


## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [ ]:
# Instala o spacy
!pip install -U spacy==2.3.5

     |████████████████████████████████| 10.4 MB 4.5 MB/s 
     |████████████████████████████████| 1.0 MB 42.0 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


## 1.9 Recupera o cohebert do Github

In [ ]:
!git clone https://github.com/osmarbraz/cohebert_v1.git

Cloning into 'cohebert_v1'...
remote: Enumerating objects: 4786, done.
remote: Counting objects: 100% (418/418), done.
remote: Compressing objects: 100% (267/267), done.
remote: Total 4786 (delta 283), reused 265 (delta 146), pack-reused 4368
Receiving objects: 100% (4786/4786), 28.78 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (3038/3038), done.


In [ ]:
#Muda o diretório corrente para a pasta clonada
import sys
sys.path.append('./cohebert_v1/cohebert')

In [ ]:
# Biblioteca cohebert
from util.utilmodulo import *
from util.utiltempo import *
from util.utilarquivo import *

# 2 Parametrização

In [ ]:
# Biblioteca cohebert
from bert.bertarguments import ModeloArgumentosMedida

# Definição dos parâmetros do Modelo para o cálculo de Medida
model_args = ModeloArgumentosMedida(     
    max_seq_len = 512,
    #pretrained_model_name_or_path = 'neuralmind/bert-large-portuguese-cased',
    pretrained_model_name_or_path = 'https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip',
    #pretrained_model_name_or_path = 'neuralmind/bert-base-portuguese-cased',
    #pretrained_model_name_or_path = 'https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip',       
    #pretrained_model_name_or_path = 'bert-base-multilingual-cased',
    modelo_spacy = 'pt_core_news_lg',
    versao_spacy = '2.3.0',
    do_lower_case = False,   # default True
    output_attentions = False,    # default False
    output_hidden_states = True, # default False
    use_wandb = False,
    salvar_modelo_wandb = False,    
    salvar_medicao = False, #Salva o resultado da medição
    salvar_avaliacao = False, # Salva o resultado da avaliação das medições
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path.
    estrategia_pooling = 0, # 0 - MEAN estratégia média / 1 - MAX  estratégia maior
    palavra_relevante = 0 # 0 - Considera todas as palavras das sentenças / 1 - Desconsidera as stopwords / 2 - Considera somente as palavras substantivas
)

# 3 spaCy

## 3.1 Carrega o modelo do spacy

https://spacy.io/models/pt

In [ ]:
# Biblioteca cohebert
from spacynlp.spacymodulo import *

# Carrega o modelo
nlp = carregaSpacy(model_args)

2021-10-16 08:22:36,768 : INFO : Download do arquivo do modelo do spaCy.
2021-10-16 08:22:37,107 : INFO : Download do arquivo: cohebert_v1/pt_core_news_lg-2.3.0.tar.gz.


  0%|          | 0.00/577M [00:00<?, ?B/s]

2021-10-16 08:22:55,346 : INFO : Descompactando o arquivo do modelo do spaCy.
2021-10-16 08:23:08,517 : INFO : spaCy carregado.


# 4 Avaliação

## 4.1 Wandb

https://wandb.ai/osmar-braz/medidacoerenciacstnews_v1/table?workspace=user-osmar-braz

### Função de inicialização wandb

In [ ]:
def inicializacaoWandb():

  if model_args.use_wandb:

    # Importando a biblioteca.
    import wandb

    #Login via linha de comando
    !wandb login aded3bc0ea651fff536cc08ba69caf8ac4141cfd

    # Inicializando o registro do experimento.
    # Na execução só pode existir de um init para que não gere dois registros no wandb.
    wandb.init(project='medidacoerenciacstnews_v1', name='medidacoerenciacstnews_v1')

    # Atualiza os parâmetros do modelo no wandb.
    wandb.config.update(model_args)

    # Registra os parämetros não literais do model_args.
    wandb.log({'max_seq_len': model_args.max_seq_len})
    wandb.log({'do_lower_case': model_args.do_lower_case})
    wandb.log({'output_hidden_states': model_args.output_hidden_states})    
    
    return wandb

## 4.2 Cálculo de todas as medidas dos modelos

- Total de Pares : 4980

**BERTimbau base**
- Pares Corretos Ccos: 3126
- Acurácia:  62.77108433734939

- Pares Corretos Ceuc: 3196
- Acurácia:  64.17670682730923

- Pares Corretos Cman: 3148
- Acurácia:  63.212851405622494

**BERTimbau base ajustado**
- Pares Corretos Ccos: 4621
- Acurácia:  92.79116465863454

- Pares Corretos Ceuc: 4828
- Acurácia:  96.94779116465864

- Pares Corretos Cman: 4835
- Acurácia:  97.08835341365462


**BERTimbau large**
- Pares Corretos Ccos: 3132
- Acurácia:  62.89156626506024

- Pares Corretos Ceuc: 3132
- Acurácia:  62.89156626506024

- Pares Corretos Cman: 3133
- Acurácia:  62.911646586345384
  

**BERT multilingual-cased**
- Pares Corretos Ccos: 2896
- Acurácia:  58.387096774193544

- Pares Corretos Ceuc: 2891
- Acurácia:  58.28629032258065

- Pares Corretos Cman: 2893
- Acurácia:  58.32661290322581

### Procedimento de cálculo de todos os modelos

In [ ]:
# Biblioteca cohebert
from conjuntodedados.dadoscstnewsmedida import *
from bert.bertmodulo import *
from experimento.calculomedida  import *

def procedimentoCalculaMedida(modelo_pretreinado, estrategia_pooling, palavra_relevante):

  # Import das bibliotecas
  import time
  import datetime

  # Seta o parâmetro do modelo pré-treinado
  model_args.pretrained_model_name_or_path = modelo_pretreinado
  # Seta o parâmetro da estratégia
  model_args.estrategia_pooling = estrategia_pooling
  # Seta o parâmetro de relevância
  model_args.palavra_relevante = palavra_relevante

  print("Processamendo do modelo pré-treinado: {} e estrategia pooling {} e com palavra relevante {}".format(modelo_pretreinado,estrategia_pooling,palavra_relevante))
      
  # Marca o tempo de início do processamento
  tempoInicioTeste = time.time()
  print("Tempo início processamento: {:} (h:mm:ss)".format(formataTempo(tempoInicioTeste)))
  
  # Carrega o Spacy
  nlp = carregaSpacy(model_args)

  # Carrega o modelo e o tokenizador do bert
  model, tokenizer = carregaBERT(model_args)
  
  # Carrega o conjunto de dados
  dfdados = getConjuntoDeDadosMedida(model_args, tokenizer, None)
  
  # Inicializa o wandb para registro
  wandb = inicializacaoWandb()

  print("antes")
  # Calcula as medidas dos documentos do conjunto de dados
  lista_medidas_documentos_salvar, conta, acuraciaCcos, contaCcos, acuraciaCeuc, contaCeuc, acuraciaCman, contaCman = calculaMedidasDocumentosConjuntoDeDados(model_args, dfdados, model, tokenizer, nlp, wandb)
  print("depois")
  
  # Pega o tempo atual menos o tempo do início do processamento.
  tempoFinalTeste = time.time()
  tempoTotalProcessamento = formataTempo(tempoFinalTeste - tempoInicioTeste)

  # Nome base do arquivo de medidas
  NOME_BASE = 'MedicaoCoerenciaCSTNews_v1'  
  # Diretório para salvar o arquivo.
  DIRETORIO_MEDICAO = '/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_medicao/Medicao/'
  # Salva o resultado da classificação
  salvaResultadoMedicao(model_args, NOME_BASE, DIRETORIO_MEDICAO, lista_medidas_documentos_salvar)

  # Nome base do arquivo de medidas
  NOME_BASE = 'MedicaoCoerenciaCSTNews_v1'  
  # Diretório para salvar o arquivo de resultado.
  DIRETORIO_AVALIACAO = '/content/drive/MyDrive/Colab Notebooks/Data/CSTNEWS/validacao_medicao/Avaliacao/'
  # Salva o resultado da avaliação
  salvaResultadoAvaliacao(model_args, NOME_BASE, DIRETORIO_AVALIACAO, tempoTotalProcessamento, conta, acuraciaCcos, contaCcos, acuraciaCeuc, contaCeuc, acuraciaCman, contaCman)

  print("Tempo processamento:  {:} (h:mm:ss)".format(tempoTotalProcessamento))

  # Apaga os objetos
  del tokenizer
  del model
  del lista_medidas_documentos_salvar

  # Esvazia o buffer dos embeddings originais
  limpaBufferEmbedding()

### Executa o procedimento para todos os parâmetros

In [ ]:
# Import das bibliotecas
from tqdm.notebook import tqdm as tqdm_notebook

# Definição dos parâmetros a serem avaliados
# Nomes dos modelos pré-treinados a serem avaliados
NOME_MODELO_PRETREINADO = ['https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip',
                           'https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip',
                           'bert-base-multilingual-cased']

# Estratégias pooling a serem avaliadas
ESTRATEGIA_POOLING = [0, 1] # (0 - MEAN, 1 - MAX)

# Estratégias de palavras relevantes das sentenças a serem avaliadas
PALAVRA_RELEVANTE_STR = ['TODAS_AS_PALAVRAS', 'SEM_STOPWORDS', 'SOMENTE_SUBSTANTIVOS']
PALAVRA_RELEVANTE = [0, 1, 2] # (0 - ALL, 1 - CLEAN, 2 - NOUN) - 'TAP,SSW,SBT'

# Barra de progresso modelos pré-treinados do BERT
modelo_pretreinado_bar = tqdm_notebook(range(0,len(NOME_MODELO_PRETREINADO)), desc=f'Modelos', unit=f'modelo')

# Percorre todos os modelo pré-treiandos do BERT a serem avaliados
for modelo_pretreinado_i in modelo_pretreinado_bar:

  # Barra de progresso estratégias de pooling
  estrategia_pooling_bar = tqdm_notebook(range(len(ESTRATEGIA_POOLING)), desc=f'Estratégias Pooling', unit=f'estratégia')

  # Percorre todos as estratégias de pooling  a serem avaliadas
  for estrategia_pooling_i in estrategia_pooling_bar:

    # Barra de progresso palavras relevantes
    palavra_relevante_bar = tqdm_notebook(range(len(PALAVRA_RELEVANTE)), desc=f'Palavras relevantes', unit=f'relavante')

    # Percorre todos as estratégias de palavras relevantes a serem avaliadas
    for palavra_relevante_i in palavra_relevante_bar:

      # Passa os parâmetros para o procedimento cálculo das medidas para o modelo, estratégia de pooling e de palavras relevantes
      procedimentoCalculaMedida(NOME_MODELO_PRETREINADO[modelo_pretreinado_i], ESTRATEGIA_POOLING[estrategia_pooling_i], PALAVRA_RELEVANTE[palavra_relevante_i])      

Modelos:   0%|          | 0/3 [00:00<?, ?modelo/s]

Estratégias Pooling:   0%|          | 0/2 [00:00<?, ?estratégia/s]

Palavras relevantes:   0%|          | 0/3 [00:00<?, ?relavante/s]

Processamendo do modelo pré-treinado: https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip e estrategia pooling 0 e com palavra relevante 0
Tempo início processamento: 18916 days, 8:23:37 (h:mm:ss)


2021-10-16 08:23:42,463 : INFO : spaCy carregado.
2021-10-16 08:23:42,761 : INFO : Download do arquivo: bert-base-portuguese-cased_pytorch_checkpoint.zip.


...

2021-10-16 17:06:21,443 : INFO : Diretório cohebert_v1/modeloBERT do modelo BERT pronta!
2021-10-16 17:06:21,631 : INFO : Usando modelo BERT pré-treinado.
2021-10-16 17:06:21,637 : INFO : Carregando o modelo BERT do diretório cohebert_v1/modeloBERT para cálculo de medidas.
2021-10-16 17:06:28,418 : INFO : Carregando o tokenizador BERT do diretório cohebert_v1/modeloBERT.
2021-10-16 17:06:28,475 : INFO : Realizando o download do CSTNews do meu OneDrive.
2021-10-16 17:06:30,120 : INFO : Download do arquivo: Summarycoherencemodels.zip.


  0%|          | 0.00/498M [00:00<?, ?B/s]

2021-10-16 17:07:16,784 : INFO : Geração de pares de documentos concluído: 5020.
2021-10-16 17:07:34,805 : INFO : Quantidade de dados anterior: 5020.
2021-10-16 17:07:34,809 : INFO : Nova quantidade de dados    : 4980.
2021-10-16 17:07:34,846 : INFO : Quantidade de registros removidos: 40.


antes


Pares documentos:   0%|          | 0/4980 [00:00<?, ?par/s]

# 5 Finalização

## 5.1 Tempo final de processamento



In [ ]:
 # Pega o tempo atual menos o tempo do início do processamento.
finalProcessamento = time.time()
tempoTotalProcessamento = formataTempo(finalProcessamento - inicioProcessamento)

print('')
print('  Tempo processamento:  {:} (h:mm:ss)'.format(tempoTotalProcessamento))

Executa o wandb para finalizar a execução anterior

In [ ]:
if model_args.use_wandb:
    # Importando a biblioteca
    import wandb

    # Inicializando o registro do experimento
    # Na execução só pode existir de um init  para que não gere dois registros no wandb.
    wandb.init(project='medidacoerenciacstnews_v1', name='medidacoerenciacstnews_v1')